In [1]:
from goes2go import GOES

In [10]:
G = GOES(satellite=17, product='ABI-L2-ACM', domain='F')
G

╭──────────────────────────
│ 🌎 GOES Object   
│ ───────────────
│  self.satellite='noaa-goes17'
│  self.product='ABI-L2-ACMF'
│  self.domain='F'
│  self.bands=None
│  self.description='Advanced Baseline Imager Level 2 Clear Sky Mask Full Disk'
╰──────────────────────────

In [19]:
G.nearesttime('2021-07-19 21:00:00')

 👮🏻‍♂️ File already exists. Do not overwrite: C:\Users\blayl_depgywe\data\noaa-goes17\ABI-L2-ACMF\2021\200\21\OR_ABI-L2-ACMF-M6_G17_s20212002100321_e20212002109388_c20212002110059.nc
📦 Finished downloading [1] files to [C:\Users\blayl_depgywe\data\noaa-goes17\ABI-L2-ACMF].
📚 Finished reading [1] files into xarray.Dataset.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

<xarray.Dataset>
Dimensions:                                                     (y: 5424,
                                                                 x: 5424,
                                                                 number_of_time_bounds: 2,
                                                                 number_of_image_bounds: 2,
                                                                 RTM_BT_comparison_bands: 2,
                                                                 number_of_LZA_bounds: 2,
                                                                 number_of_SZA_bounds: 2)
Coordinates: (12/15)
    t                                                           datetime64[ns] ...
  * y                                                           (y) float32 0...
  * x                                                           (x) float32 -...
    y_image                                                     float32 ...
    x_image                                                     float32 ...
    retrieval_local_zenith_angle                                float32 ...
    ...                                                          ...
    RTM_BT_comparison_wavelengths                               (RTM_BT_comparison_bands) float32 ...
    RTM_BT_comparison_band_ids                                  (RTM_BT_comparison_bands) int8 ...
    dataset_name                                                <U72 'OR_ABI-...
    date_created                                                <U22 '2021-07...
    time_coverage_start                                         <U22 '2021-07...
    time_coverage_end                                           <U22 '2021-07...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                RTM_BT_comparison_bands, number_of_LZA_bounds,
                                number_of_SZA_bounds
Data variables: (12/38)
    BCM                                                         (y, x) float32 ...
    DQF                                                         (y, x) float32 ...
    time_bounds                                                 (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                                      int32 ...
    y_image_bounds                                              (number_of_image_bounds) float32 ...
    x_image_bounds                                              (number_of_image_bounds) float32 ...
    ...                                                          ...
    quantitative_local_zenith_angle_bounds                      (number_of_LZA_bounds) float32 ...
    retrieval_solar_zenith_angle_bounds                         (number_of_SZA_bounds) float32 ...
    twilight_solar_zenith_angle_bounds                          (number_of_SZA_bounds) float32 ...
    percent_uncorrectable_GRB_errors                            float32 ...
    percent_uncorrectable_L0_errors                             float32 ...
    filename                                                    <U108 'noaa-g...
Attributes: (12/26)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    processing_level:          National Aeronautics and Space Administration ...
    cdm_data_type:             Image
    timeline_id:               ABI Mode 6
    production_data_source:    Realtime
    id:                        acaac04f-ab8f-4c4a-82c8-5129bac469d3
    path:                      ['noaa-goes17/ABI-L2-ACMF/2021/200/21/OR_ABI-L...

In [20]:
G.timerange('2021-07-19 21:00', '2021-07-20 00:00')

 👮🏻‍♂️ File already exists. Do not overwrite: C:\Users\blayl_depgywe\data\noaa-goes17\ABI-L2-ACMF\2021\200\21\OR_ABI-L2-ACMF-M6_G17_s20212002100321_e20212002109388_c20212002110059.nc
📦 Finished downloading [18] files to [C:\Users\blayl_depgywe\data\noaa-goes17\ABI-L2-ACMF].


,file,product_mode,satellite,start,end,creation
0,noaa-goes17/ABI-L2-ACMF/2021/200/21/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 21:00:32.100,2021-07-19 21:09:38.800,2021-07-19 21:10:05.900
1,noaa-goes17/ABI-L2-ACMF/2021/200/21/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 21:10:31.900,2021-07-19 21:19:38.600,2021-07-19 21:20:08.100
2,noaa-goes17/ABI-L2-ACMF/2021/200/21/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 21:20:31.900,2021-07-19 21:29:38.500,2021-07-19 21:30:02.800
3,noaa-goes17/ABI-L2-ACMF/2021/200/21/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 21:30:31.900,2021-07-19 21:39:38.600,2021-07-19 21:40:05.000
4,noaa-goes17/ABI-L2-ACMF/2021/200/21/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 21:40:31.900,2021-07-19 21:49:38.600,2021-07-19 21:50:06.600
5,noaa-goes17/ABI-L2-ACMF/2021/200/21/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 21:50:31.900,2021-07-19 21:59:38.600,2021-07-19 22:00:02.500
6,noaa-goes17/ABI-L2-ACMF/2021/200/22/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 22:00:31.900,2021-07-19 22:09:38.600,2021-07-19 22:10:03.800
7,noaa-goes17/ABI-L2-ACMF/2021/200/22/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 22:10:31.900,2021-07-19 22:19:38.600,2021-07-19 22:20:06.800
8,noaa-goes17/ABI-L2-ACMF/2021/200/22/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 22:20:31.900,2021-07-19 22:29:38.600,2021-07-19 22:30:00.900
9,noaa-goes17/ABI-L2-ACMF/2021/200/22/OR_ABI-L2-...,ABI-L2-ACMF-M6,G17,2021-07-19 22:30:31.900,2021-07-19 22:39:38.600,2021-07-19 22:40:02.100
